# Advanced features

In [1]:
# Add BipartitePandas to system path, do not run this
# import sys
# sys.path.append('../../..')

## Import the BipartitePandas package

Make sure to install it using `pip install bipartitepandas`.

In [2]:
import bipartitepandas as bpd

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Get your data ready

For this notebook, we simulate data.

In [3]:
df = bpd.SimBipartite().simulate()
bdf = bpd.BipartiteDataFrame(i=df['i'], j=df['j'], y=df['y'], t=df['t'])
display(bdf)

,i,j,y,t
0,0,91,-0.044204,0
1,0,91,-0.885095,1
2,0,92,-0.810008,2
3,0,94,-1.144337,3
4,0,22,-2.223591,4
...,...,...,...,...
49995,9999,144,0.849494,0
49996,9999,144,1.292909,1
49997,9999,144,1.558122,2
49998,9999,144,4.055746,3


## Advanced data cleaning

*Hint:* want details on all cleaning parameters? Run `bpd.clean_params().describe_all()`, or search through `bpd.clean_params().keys()` for a particular key, and then run `bpd.clean_params().describe(key)`.

#### Compute the largest connected component

Use the parameter `connectedness` to set the connectedness measure to use, and `component_size_variable` to choose how to measure component size (e.g. `firms` chooses the connected component with the greatest number of unique firm ids, while `workers` chooses the connected component with the greatest number of unique worker ids).

*Note:* connectedness is NOT NECESSARILY maintained between non-collapsed and collapsed formats. Therefore, if you plan to use connected, collapsed data, it is recommended to first clean the non-collapsed data with `connectedness=None`, next collapse the data, and finally clean the collapsed data with the connectedness measure you plan to use.

#### Set how to handle worker-year duplicates

Use the parameter `i_t_how` to customize how worker-year duplicates are handled.

#### Collapse at the match-level

If you drop the `t` column, collapsing will automatically collapse at the match level. However, this prevents conversion to event study format (this can be bypassed with the `.construct_artificial_time()` method, but the data will likely have a meaningless order, rendering the event study uninterpretable).

#### Avoid unnecessary copies

If you are working with a large dataset, you will want to avoid copies whenever possible. So set `copy=False`.

#### Avoid unnecessary sorts

If you know your data is sorted by `i` and `t` (or, if you aren't including a `t` column, just by `i`), then set `is_sorted=True`.

#### Avoid complicated loops

Sometimes workers leave a firm, then return to it (we call these workers *returners*). Returners can cause computational difficulties because sometimes intermediate firms get dropped (e.g. a worker goes from firm $A \to B \to A$, but firm $B$ gets dropped). This turns returners into stayers. This can change the largest connected set of firms, and if data is in collapsed format, requires the data to be recollapsed.

Because of these potential complications, if there are returners, many methods require loops that run until convergence.

These difficulties can be avoided by setting the parameter `drop_returns` (there are multiple ways to handle returners, they can be seen by running `bpd.clean_params().describe('drop_returns')`).

*Alternative:* another way to handle returners is to drop the `t` column. Then, sorting will automatically sort by `i` and `j`, which eliminates the possibility of returners. However, this prevents conversion to event study format (this can be bypassed with the `.construct_artificial_time()` method, but the data will likely have a meaningless order, rendering the event study uninterpretable).

#### Disable logging

Logging can slow down data cleaning. Set the parameter `log=False` when constructing your dataframe to turn off logging.

## Restoring original ids

To restore original ids, we need to make sure the dataframe is tracking ids as they change.

We make sure the dataframe tracks ids as they change by setting `include_id_reference_dict=True`.

Notice that in this example we use `j / 2`, so that `j` will be modified during data cleaning.

In [4]:
bdf_original_ids = bpd.BipartiteDataFrame(i=df['i'], j=(df['j'] / 2), y=df['y'], t=df['t'], include_id_reference_dict=True).clean()
display(bdf_original_ids)

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how=False)
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how=None)
sorting columns
resetting index


,i,j,y,t,m
0,0,0,-0.044204,0,0
1,0,0,-0.885095,1,1
2,0,1,-0.810008,2,2
3,0,2,-1.144337,3,2
4,0,3,-2.223591,4,1
...,...,...,...,...,...
49995,9999,81,0.849494,0,0
49996,9999,81,1.292909,1,0
49997,9999,81,1.558122,2,0
49998,9999,81,4.055746,3,1


#### Merging in original ids

The method `.original_ids()` will return a dataframe that merges in the original ids.

In [5]:
bdf_original_ids.original_ids()

,i,j,y,t,m,original_j
0,0,0,-0.044204,0,0,45.5
1,0,0,-0.885095,1,1,45.5
2,0,1,-0.810008,2,2,46.0
3,0,2,-1.144337,3,2,47.0
4,0,3,-2.223591,4,1,11.0
...,...,...,...,...,...,...
49995,9999,81,0.849494,0,0,72.0
49996,9999,81,1.292909,1,0,72.0
49997,9999,81,1.558122,2,0,72.0
49998,9999,81,4.055746,3,1,72.0


## Filling in missing years as unemployed

The method `.fill_periods` (for *Long* format) will fill in rows for missing intermediate periods. Note that this method will not work with custom columns.

In this example, we drop periods 1-3, then fill them in:

In [6]:
bdf_missing = bdf[(bdf['t'] == 0) | (bdf['t'] == 4)].clean()
display(bdf_missing.fill_periods())

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how=False)
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how=None)
sorting columns
resetting index


,i,j,y,t,m
0,0,91,-0.044204,0,1
1,0,-1,NaN,1,NaN
2,0,-1,NaN,2,NaN
3,0,-1,NaN,3,NaN
4,0,22,-2.223591,4,1
...,...,...,...,...,...
49995,9999,144,0.849494,0,1
49996,9999,-1,NaN,1,NaN
49997,9999,-1,NaN,2,NaN
49998,9999,-1,NaN,3,NaN


## Getting extended event study dataframes

BipartitePandas allows you to use *Long* format data to generate event studies with more than 2 periods.

You can specify:

- what column signals a transition (e.g. if `j` is used, a transition is when a worker moves firms)
- which column(s) should be treated as the event study outcome
- how many periods before and after the transition should be considered
- whether the pre- and/or post-trends must be stable, and for which column(s)

We consider an example where `j` is the transition column, `y` is the outcome column, and with pre- and post-trends of length 2 that are required to be at the same firm. Note that `y_f1` is the first observation after the individual moves firms.

In [7]:
es_extended = bdf.get_extended_eventstudy(transition_col='j', outcomes='y', periods_pre=2, periods_post=2, stable_pre='j', stable_post='j')
display(es_extended)

,i,t,y_l2,y_l1,y_f1,y_f2
0,5,2,0.047225,0.848582,1.379998,0.668513
1,10,2,-0.109625,-1.757969,-2.463662,-1.438843
2,11,3,2.720527,-0.156588,-0.715961,2.032219
3,12,3,-1.066937,-0.087966,-1.533458,-1.682028
4,13,2,0.637000,-0.485941,-1.024146,1.509601
...,...,...,...,...,...,...
2553,9974,2,2.349189,2.780799,1.564887,3.740422
2554,9977,3,-1.925892,-2.604099,-0.571768,-2.908175
2555,9978,2,0.576860,1.708630,2.591128,2.314370
2556,9988,3,0.946022,1.058703,-0.689564,-0.165001
